In [1]:
import json
import requests
import pandas as pd
import io
import boto3

# Step 1 Read Meta Data  & End Points 

In [2]:
def get_ids(start, end):
    years = [str(x) for x in range(start, end+1)]
    ids = {"General": [], "Research": [], "Ownership": []}
    url = 'https://openpaymentsdata.cms.gov/api/1/metastore/schemas/dataset/items?show-reference-ids'
    data = requests.get(url).json()
    for d in data:
        split_title = d['title'].split(" ")
        identifier = d['identifier']
        if split_title[0] in years:
            if split_title[1] in ids:
                ids[split_title[1]].append(identifier)
    return ids

In [3]:
ids = get_ids(2020, 2021)

In [4]:
usecols=["Date_of_Payment", "Product_Category_or_Therapeutic_Area_1", "Recipient_State", "Recipient_City", "Covered_Recipient_Type", "Total_Amount_of_Payment_USDollars"]

In [15]:
def move_data_to_s3(id_list):
    for category, category_ids in id_list.items():
        for category_id in category_ids:
            url = 'https://openpaymentsdata.cms.gov/api/1/metastore/schemas/dataset/items/{}'.format(category_id)
            data = requests.get(url).json()
            download = data["distribution"][0]["downloadURL"]
            file_name = data["title"]
            try:
#                 response = requests.get(download)
#                 df = pd.read_csv(io.BytesIO(response.content), nrows=10)
                # USE KEY TO DETERMINE WHICH S3 bucket to place data in
                # convert df to csv and name it file_name, should index = False?
                # place csv in s3
            except Exception as e:
                print("Failed to download:", download, "Error:", e)

In [16]:
move_data_to_s3(ids)

2020 General Payment Data
2021 General Payment Data
2021 Research Payment Data
2020 Research Payment Data
2021 Ownership Payment Data
2020 Ownership Payment Data
